In [2]:
!pip3 install clickhouse-connect
!pip3 install bdt
!pip3 install dbt-clickhouse

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.2/185.2 kB 519.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 600.9 kB/s eta 0:00:00a 0:00:01
  Created wheel for bdt: filename=BDT-0.0.3-py3-none-any.whl size=6122 sha256=8c066d37c86a379ab99b961a210d5ef90740a699ce288a3ffb417868dd088126
  Stored in directory: /Users/evgeniu/Library/Caches/pip/wheels/46/b9/8c/20965cb2470f761201c93c95c84c8bf46cda22f1395b215093
  Created wheel for boolexp: filename=boolexp-0.1-py3-none-any.whl size=3545 sha256=4e3c45419384fc9caabdd462d5b28dc024bd0a17c2c150d55b8b25ce30e89db2
  Stored in directory: /Users/evgeniu/Library/Caches/pip/wheels/4b/53/cd/188ec08b9b7185d739f47c7ebf4948252c454ca685179b377b
Successfully built bdt boolexp


In [2]:
import clickhouse_connect
import os

In [3]:
client = clickhouse_connect.get_client(host='clickhouse.evgeni.keenetic.link', interface='https', port=443, username='artsofte', password='Ur*2nLa=', verify=False)

In [4]:
client.command('CREATE DATABASE IF NOT EXISTS test_db')

In [5]:
with open('/Users/evgeniu/de_clickhouse.sql', 'r') as f:
    sql_commands = f.read().split('\n\n')
#print(sql_command)

In [75]:
for command in sql_commands:
    client.command(command)

Описание структуры таблиц

manager_departments - таблица содержащая информацию о принадлежности менеджера к отделу
* email - email адрес менеджера
* department - отдел к которому относится менеджер

payments - таблица содержащая информацию о поступлениях 
* id - уникальный идентификатор поступления
* value - сумма поступления 
* client_id - идентификатор клиента осуществивший платеж
* client_name - название клиента осуществивший платеж
* payment_date - дата поступления
* manager_name - ФИО менеджера ответственного за клиента
* manager_email - email адрес менеджера ответственного за клиента


#### Task 1
На основе таблицы с поступлениями, необходимо найти платежи с максимальными суммами всех клиентов которые заплатили в январе 2023 года. Результат должен быть представлен в виде таблицы со следующими полями:
Название клиента
Дата когда пришел максимальный платеж
Сумма максимального поступления

In [6]:
client.query_df('''
                SELECT client_name, payment_date, value 
                FROM
                    (SELECT client_name, payment_date, value, max(value) over (partition by client_name) as max_value
                    FROM payments
                    WHERE DATE_TRUNC('month', payment_date) = '2023-01-01')
                WHERE value = max_value
                ''')

,client_name,payment_date,value
0,Клиент #102,2023-01-20,621500.0
1,Клиент #103,2023-01-20,575600.0
2,Клиент #109,2023-01-10,878000.0
3,Клиент #111,2023-01-18,711194.0
4,Клиент #119,2023-01-31,699800.0
5,Клиент #124,2023-01-13,515660.0
6,Клиент #15,2023-01-19,595850.0
7,Клиент #153,2023-01-27,521609.0
8,Клиент #154,2023-01-19,580728.0
9,Клиент #155,2023-01-31,770000.0


#### Task 2
Необходимо определить сколько принес денег каждый отдел. Таблица с поступлениями хранит информацию с имением и email адресе менеджера. Информация об отделе хранится в отдельной таблице-справочнике manager_departments. 

Связь этих двух таблиц осуществляется по полю email, но формат почты отличается. В каких-то случаях email менеджера представлен в формате n.ssssssss@domain.ru, где n это первая буква имени, ssssssss фамилия менеджера и между ними “.” (знак точки). В каких-то случаях формат имеет вид nssssssss@domain.ru, без точки между именем и фамилией. 
Необходимо объединить две таблицы с предварительным приведением почты к единому виду и дальнейшей агрегацией. 

Если для менеджера не определен отдел (указано значение NULL), то таких менеджеров необходимо отнести в группу “Отдел не определен”


In [7]:
client.query_df('''
                SELECT md.department, SUM(payments.value)
                FROM payments JOIN (SELECT replaceOne(email, '.', '') as email, COALESCE(department, 'Отдел не определен') as department 
                                    FROM manager_departments) AS md 
                              ON payments.manager_email = md.email
                GROUP BY md.department
                ''')

,department,sum(value)
0,Отдел продаж,100970750.0
1,Партнёрка,83105623.0
2,Клиентский сервис,84710891.0
3,Клиентский сервис,118959628.0
4,ТОП-менеджмент,11995750.0
5,Отдел не определен,4741058.0
6,Проектный офис,9115198.0


#### Task 3
Необходимо дополнить таблицу с поступлениями полем “состояние клиента”. Если платеж для клиента является первым, то состояние клиента должно быть “Новый”. Для всех остальных платежей данного клиента, состояние должно быть “Действующий”.

In [8]:
client.query_df('''
                SELECT id, value, client_id, client_name, payment_date, manager_name, manager_email, case WHEN case_number = 1 THEN 'Новый'
                      WHEN case_number > 1 THEN 'Действующий' end as client_status
                FROM
                    (SELECT *, ROW_NUMBER() OVER (PARTITION BY client_id ORDER BY payment_date) AS case_number FROM payments)
                ''')

,id,value,client_id,client_name,payment_date,manager_name,manager_email,client_status
0,646,731210.0,1,Клиент #1,2023-10-03,Manager #12,nmanager12@profitbase.ru,Новый
1,600,581000.0,2,Клиент #2,2023-09-19,Manager #18,nmanager18@profitbase.ru,Новый
2,72,562640.0,3,Клиент #3,2023-02-08,Manager #23,nmanager23@profitbase.ru,Новый
3,485,562640.0,3,Клиент #3,2023-08-01,Manager #23,nmanager23@profitbase.ru,Действующий
4,218,567500.0,4,Клиент #4,2023-04-12,Manager #1,nmanager1@profitbase.ru,Новый
...,...,...,...,...,...,...,...,...
647,448,590000.0,267,Клиент #267,2023-07-19,Manager #23,nmanager23@profitbase.ru,Новый
648,491,680000.0,267,Клиент #267,2023-08-01,Manager #23,nmanager23@profitbase.ru,Действующий
649,612,581000.0,268,Клиент #268,2023-09-21,Manager #17,nmanager17@profitbase.ru,Новый
650,391,581000.0,269,Клиент #269,2023-06-23,Manager #14,nmanager14@profitbase.ru,Новый


#### Task 4

Необходимо сформировать таблицу в которой будет отражено сколько поступило денег в каждом месяце и как росла общая выручка с учетом предыдущих месяцев.

In [9]:
client.query_df('''
                SELECT *, SUM(revenue) OVER (ORDER BY period) AS revenue_cumulative
                FROM(SELECT DATE_TRUNC('month', payment_date) AS period, SUM(value) AS revenue
                    FROM payments
                    GROUP BY period)
                ''')

,period,revenue,revenue_cumulative
0,2023-01-01,30960654.0,30960654.0
1,2023-02-01,37519548.0,68480202.0
2,2023-03-01,55637744.0,124117946.0
3,2023-04-01,44849811.0,168967757.0
4,2023-05-01,41863267.0,210831024.0
5,2023-06-01,50931151.0,261762175.0
6,2023-07-01,44981771.0,306743946.0
7,2023-08-01,49063849.0,355807795.0
8,2023-09-01,52216351.0,408024146.0
9,2023-10-01,5574752.0,413598898.0


In [14]:
client.query_df('SHOW DATABASES')

,name
0,INFORMATION_SCHEMA
1,default
2,information_schema
3,system
4,test_db


In [13]:
client.query_df('DROP TABLE task1')

""
